# Git - Control de Versiones
*Sofía García Salas*

*2021-03-02*

***

Estas son mis notas sobre la clase de MIT [The Missing Semester](https://missing.csail.mit.edu/2020/version-control/).

## Control de versiones
`Git` es un sistema de control de versiones que permite documentar los cambios a código u otro documentos y folders. También facilitan el trabajo en equipo.

Estos sistemas almacenan la historia de cambios tomando una fotografía al estado de los archivos y folders. No solo tienen la historia de los cambios a estos archivos sino que también almacenan metadata (e.g. quién realizó un cambio, en qué momento).

### Utilidad
Sirve no solo en proyectos grandes, sino en proyecto personales para poder revisar versiones antiguos del código, ver por qué se cambio algo (viendo commit messages), trabajar en paralelo con distintas branches para el desarrollo, etc. 

### Git
Se ha vuelto el sistema de control de versiones más popular.

Se interfaz no es muy amigable con el usuario, por lo que el profesor recomienda empezar aprendiendo su diseño e ideas. Al entender esto, será más fácil utilizar la interfaz y resolver problemas.

## Modelo de historia de Git
Es una colección de archivos y folders dentro de un directorio maestro.
Un folder es denominado como `tree` y un archivo como `blob`

```
root
|-foo (tree)
| |-bar.txt (blob)
|-baz.txt

```